In [8]:
!pip install tensorflow

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Detekcja katastrof na podstawie tweetów z Twittera
Dlaczego analizowanie tekstu z mediów społecznościowych może być dla nas użyteczne? Bo znajduje się tam mnóstwo informacji o emocjach i opiniach ludzi. W wielu sytuacjach odpowiedzi na interesujęace nas pytania znajdują się w słowach ludzi. Przykładowo jakaś firma mogłaby chcieć bez wkładania wysiłku w badania poznać opinie o swoim produkcie - czy ludziom się podoba, na co narzekają, czy u jakiejś większej grupy zaczynają pojawiać się usterki spowodowane być może błędem w produkcji itp. 

Problemem jest jednak fakt, że tekst trudno jest nam reprezentować. Trudno zakodować go w postaci, z której będzie można odczytać jego prawdziwe znaczenie. Z pomocą przychodzą nam na szczęście metody przetwarzania języka naturalnego.

W tym notebooku postaram się w automatyczny sposób sprawdzać czy dany tweet zawiera informację o katastrofie. Czasem zdarza się, że analiza takich danych może w szybszy sposób zauważyć na przykład trzęsienia ziemi niż naukowcy wyposażeni w odpowiednie przyrządy (trzęsięnie ziemi w Sichuan, Chiny, 2008)

## Dane
Nasze dane są w postaci tsv (tab separated values). Zawierają trzy kolumny - tekst, kolumnę choose_one, która zawiera informaję czy tweet jest o jakiejś katastrofie i class_label - 0 lub 1. Istnieje też opcja, że osoba która klasyfikowała ręcznie te dane nie była w stanie stwierdzić do jakiej kategorii należy tweet. Wtedy choose_one jest 'Can't Decide', a class_label wynosi 2.

In [9]:
import numpy as np
import pandas as pd

disasters_dataset = pd.read_table('../notebooks/data/disasters_socialmedia.tsv')
disasters_dataset.head()

,text,choose_one,class_label
0,Just happened a terrible car crash,Relevant,1
1,Our Deeds are the Reason of this #earthquake M...,Relevant,1
2,"Heard about #earthquake is different cities, s...",Relevant,1
3,"there is a forest fire at spot pond, geese are...",Relevant,1
4,Forest fire near La Ronge Sask. Canada,Relevant,1


W naszym zbiorze danych jest trochę więcej danych z kategorii 'tweety zwyczajne' niż 'tweety katastrofalne'.

In [10]:
disasters_dataset.groupby('class_label')['text'].count()

class_label
0    6187
1    4673
2      16
Name: text, dtype: int64

In [11]:
disasters_dataset.choose_one.unique()

array(['Relevant', 'Not Relevant', "Can't Decide"], dtype=object)

Poniżej jest lista tweetów, w których osoby nie mogły zdecydować się do jakiej kategorii dany tweet należy.

In [12]:
for i,text in enumerate(disasters_dataset[disasters_dataset['class_label']==2]['text']): print('#{}'.format(i), text)

#0 Why is there an ambulance right outside my work
#1 @MisfitRarity misfit got bombed
#2 @RockBottomRadFM Is one of the challenges on Tough Enough rescuing people from burning buildings?
#3 ? High Skies - Burning Buildings ? http://t.co/uVq41i3Kx2 #nowplaying
#4 What if we used drones to help firefighters lead people out of burning buildings/ help put the fire out?
#5 San Bernardino I10 W Eo / Redlands Blvd **Trfc Collision-No Inj** http://t.co/FT9KIGmIgh
#6 Kinetic Typography Crash Course (After Effects) (Video) http://t.co/fL8gCi84Aj #course http://t.co/dVONWIv3l1
#7 Deaths 5 http://t.co/0RtxTT11jj
#8 @MythGriy they can't detonate unless they touch the ground
#9 MPD director Armstrong: when this first happened I cannot begin to tell you the devastation I felt.
#10 Displaced Persons GN (2014 Image) #1-1ST NM http://t.co/yEJt18sbm0 http://t.co/RcqacN91bE
#11 Large rain drops falling in Rock Hill off Anderson Road. #rain #scwx #drought
#12 I See Fire
#13 it's actually funny how chihaya 

In [13]:
disasters_dataset = disasters_dataset.drop('choose_one', axis=1)

In [14]:
import random

disasterous = disasters_dataset[disasters_dataset['class_label']==1]
casual = disasters_dataset[disasters_dataset['class_label']==0]

print('Fraction of casual tweets: {:.3f}'.format(len(casual)/(len(disasterous)+len(casual))))
print('\n==================\nDISASTEROUS TWEETS\n==================')
for tweet in random.sample(list(disasterous.text), 5):
    print('------\n',tweet)
print('\n==================\nCASUAL TWEETS\n==================')
for tweet in random.sample(list(casual.text), 5):
    print('------\n',tweet)

Fraction of casual tweets: 0.570

DISASTEROUS TWEETS
------
 NetNewsLedger Wild Fire Update  - August 4 2015: THUNDER BAY  - WEATHER  - Thereξwere no new fires confirmed by t... http://t.co/JflxgEmBdA
------
 @freddiedeboer @Thucydiplease then you have rise of Coates Charleston massacre Walter Scott and black twitter more broadly as well.
------
 The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/dOFRh5YB01
------
 CHS issues Hazardous Weather Outlook (HWO)  http://t.co/lbkiyfwFlU #WX
------
 Earthquake : M 3.4 - 96km N of Brenas Puerto Rico: Time2015-08-05 10:34:24 UTC2015-08-05 06:34:24 -04:00 at ﾗ_ http://t.co/sDZrrfZhMy

CASUAL TWEETS
------
 Greece's tax revenues collapse as debt crisis continues: As talks continue over proposed  �86bn third bailout ... http://t.co/7w2WiEFjuq
------
 #yyc #hailstorm #christmas came early https://t.co/f0A2IIzx3A
------
 @StevenOnTwatter @PussyxDestroyer just order a blizzard pay then put your nuts in it say they hav

W dalszej części będę korzystał z biblioteki keras
(https://keras.io/#you-have-just-found-keras)

https://keras.io/preprocessing/text/

## Reprezentacja tekstu

In [20]:
from collections import namedtuple
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True,
                      split=' ',
                      char_level=False)

tokenizer.fit_on_texts(disasters_dataset.text)

EncodedDataset = namedtuple('EncodedDataset', ('instances', 'labels'))

texts_enco = tokenizer.texts_to_sequences(disasters_dataset.text)
texts_padded = sequence.pad_sequences(texts_enco, padding='pre')
encoded_dataset = EncodedDataset(instances=texts_padded, labels=disasters_dataset['class_label'])
vocab_size = len(tokenizer.word_index) + 1
sequence_len = len(texts_padded[0])


In [30]:
len(encoded_dataset.instances[3])

33

In [33]:
max([len(x) for x in texts_enco])

33

To co zrobiliśmy to zamieniliśmy tworzyliśmy tokenizer, który uczy się na naszym korpusie tweetów. Wywołując metodę fit_on_texts tokenizer zapamiętał wszystkie słowa jakie kiedykolwiek zostały użyte (zbiór tych słów będę nazywał słownikiem). Następnie w texts_enco umieściliśmy poszczególne tweety jako ciągi wektorów długości tweeta (w słowach) w którym na k-tym miejscu jest numer k-tego słowa w słowniku. Tweety na szczęście są niedługie (najdłuższy ma długość 33), więc możemy sprawić, że każdy wektor będzie miał długość 33. Nową reprezentację tweetów umieszczamy w texts_padded (jeżeli tweet jest krótszy niż 33 słowa to na poprzednich miejscach ma zera).

In [34]:
for instance in encoded_dataset.instances[:10]:
    print(instance)

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0   33
  796    5 1520  132   98]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0  115 5930   24    4  858    8   21  254
  153 1823 3839   89   42]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0  384   55  254   11
 1320 1824  660 1521  275]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
   78   11    5  170   44   20  834 2545 8782   24 4613  902    4  691
   10 1383  495  101   42]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0  170   44
  214  903 8783 8784 1447]
[   0    0    0    0    0    0    0    0    0    0    0   42 1609 1522
    6 8785    7 5931   24  144 8786   19 1825   40  283  279   58 2282
    7  797 17

Uwaga: ta metoda ma oczywiście swoje wady. Nie bierzemy pod uwagę w żaden sposób kolejności występowania słów i nie nadajemy im żadnego znaczenia. Można jednak oczekiwać, że będziemy w stanie z pewnym prawdopodobieństwem wykrywać tweety o katastrofach na podtawie samego tylko występowania poszczególnych słów.

## Podział na zbiór treningowy i testowy

In [37]:
random.seed(0)
def get_splits(dataset, valid_fraction=0.25):
    data_size = len(dataset.instances)
    split_id = int(data_size * valid_fraction)
    merged_data = list(zip(dataset.instances, dataset.labels))
    random.shuffle(merged_data)
    shuffled_instances, shuffled_labels = zip(*merged_data)
    valid_data = EncodedDataset(instances=shuffled_instances[0:split_id],
                                labels=shuffled_labels[0:split_id])
    train_data = EncodedDataset(instances=shuffled_instances[split_id:],
                                labels=shuffled_labels[split_id:])
    return train_data, valid_data

train_data, valid_data = get_splits(encoded_dataset)

print('Training data size: {}, Validation data size: {}'.format(len(train_data.labels), len(valid_data.labels)))

Training data size: 8157, Validation data size: 2719


## Budowanie modelu

Dzięki kerasowi budowanie modelu jest bardzo proste.

In [44]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Embedding
from keras.optimizers import SGD

tf.set_random_seed(0)
np.random.seed(0)

print('Building model...', end=' ')
model = Sequential()
model.add(Embedding(input_length=sequence_len, input_dim=vocab_size, output_dim=50, mask_zero=True)) 
# Set mask_zero=False when using CNN
model.add(SimpleRNN(64))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=SGD(), metrics=['accuracy'])
print('done')

print('Training model..')
x_train = np.array(train_data.instances, np.float32)
y_train = np.array(train_data.labels, np.int32)
x_valid = np.array(valid_data.instances, np.float32)
y_valid = np.array(valid_data.labels, np.int32)
model.fit(x=x_train, y=y_train, validation_data=(x_valid, y_valid), batch_size=32, epochs=5, verbose=1)
print('Training comleted')

Building model... done
Training model..
Train on 8157 samples, validate on 2719 samples
Epoch 1/5
8157/8157 [==============================] - 17s 2ms/step - loss: 0.6711 - acc: 0.5961 - val_loss: 0.6589 - val_acc: 0.6278
Epoch 2/5
8157/8157 [==============================] - 15s 2ms/step - loss: 0.6488 - acc: 0.6300 - val_loss: 0.6466 - val_acc: 0.6289
Epoch 3/5
8157/8157 [==============================] - 16s 2ms/step - loss: 0.6370 - acc: 0.6431 - val_loss: 0.6329 - val_acc: 0.6488
Epoch 4/5
8157/8157 [==============================] - 14s 2ms/step - loss: 0.6203 - acc: 0.6686 - val_loss: 0.6244 - val_acc: 0.6587
Epoch 5/5
8157/8157 [==============================] - 14s 2ms/step - loss: 0.5971 - acc: 0.6889 - val_loss: 0.6214 - val_acc: 0.6679
Training comleted


In [45]:
print("Final validation..")
_, acc = model.evaluate(x_valid, y_valid, verbose=1) 
print("Accuracy on validation set: {:.3f}".format(acc))

Final validation..
2719/2719 [==============================] - 1s 441us/step
Accuracy on validation set: 0.668


Skuteczność na poziomie 0.67 jest lepsza niż całkiem losowa, ale nadal niezbyt satysfakcjonująca. Będzie trzeba spróbować ją poprawić eksperymentując z innymi modelami (być może bardziej skom